# Essentials

In [237]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from config.fucntions import columns_converter as cc
from config.fucntions import img_downloader as imgd
import json
from time import sleep
import numpy as np
from concurrent.futures import ThreadPoolExecutor


def percentage(a, b):
    return round(a/b*100, 3)

In [238]:
df_mvp = pd.read_csv('datasets/production/dataset_MVP_clustered.csv', sep=';', encoding='utf-8')
df_mvp_buildings = df_mvp[(df_mvp.cluster_type =='outdoor') | (df_mvp.cluster_type =='indoor') | (df_mvp.cluster_type =='details') | (df_mvp.cluster_type =='people')]

df_clean = pd.read_csv('datasets/production/dataset_CLEAN_UPDATED.csv', sep=';', encoding='utf-8')

cc.strings_to_links(df_mvp_buildings, df_mvp_buildings.columns[1:2])
cc.strings_to_links(df_mvp, df_mvp.columns[1:2])

a = []
for file in df_mvp_buildings.file_name:
    a.append(imgd.filename_to_objects(df_mvp_buildings, file)[0])

a = list(set(a))
df_buildings = df_clean.iloc[a]

# Original cleaned dataset with all information about objects with some metadata already filled in
df_images = pd.read_csv('datasets/production/dataset_CLEAN_UPDATED.csv', sep=';', encoding='utf-8')
df_images = df_images.rename(columns={'dated': 'Datierung', 'obj_type': 'Gattung', 'location': 'Standort'})
df_images.head(4)

,id_record,id_persistent,label_id,image_links,archive_links,microfiche_links,microfiche_archive_links
0,http://www.bildindex.de/document/obj20666124,http://id.bildindex.de/thing/0001618283,0,[],[],['http://www.bildindex.de/bilder/d/mi12308g07'],['https://www.bildindex.de/media/obj20666124/m...
1,http://www.bildindex.de/document/obj20727600,http://id.bildindex.de/thing/0001677732,1,['http://www.bildindex.de/bilder/d/fm140030'],['https://www.bildindex.de/media/obj20727600/f...,['http://www.bildindex.de/bilder/d/mi02117a05'...,['https://www.bildindex.de/media/obj20727600/m...
2,http://www.bildindex.de/document/obj20943303,http://id.bildindex.de/thing/0001803584,2,['http://www.bildindex.de/bilder/d/STMP_II_110...,['https://www.bildindex.de/media/obj20943303/S...,[],[]
3,http://www.bildindex.de/document/obj20948462,http://id.bildindex.de/thing/0001806337,3,['http://www.bildindex.de/bilder/d/STMP_II_156...,['https://www.bildindex.de/media/obj20948462/S...,[],[]


# Extraction of metadata from HTML 
(only if you don't have `metadata_idrecord_GSD.csv`)

Creating new columns for metadata scrapping

In [239]:
metadata_labels = ["Gattung", "Standort", "Datierung"]

In [240]:
df_images.shape, df_images.id_record.unique().shape

((34712, 7), (27993,))

`id_record` is not unique for each image, thus we can reduse scrapping time by going through unique `id_record`s rather than whole DataFrame. Let's create a separate dataset for it

In [241]:
# metadata = pd.DataFrame(columns=['id_record', 'Gattung', 'Standort', 'Datierung'])
# metadata['id_record'] = df_images.id_record.unique()
# metadata

In [242]:
# failed_requests = []

In [243]:
#  <----------- Uncomment to run the script --------------->

# start_ind = metadata[metadata.Gattung.isna() & metadata.Standort.isna() & metadata.Datierung.isna()].index[0]
# i = 0

# for indx in metadata.id_record.index[start_ind:]:

#     i+=1
    
#     if i % 1000 == 0:
#         metadata.to_csv('datasets/production/metadata_idrecord_GSD.csv', sep=';', encoding='utf-8', index=False)

#     if metadata.iloc[indx][['Gattung', 'Standort', 'Datierung']].isna().sum() < 3:
#         continue

#     url = metadata.id_record.loc[indx]

#     # # For speeding up scrapping only on most relevant objects (realated to building)
#     # if url not in df_buildings.id_record.values:
#     #     continue

#     try:

#         response = requests.get(url, timeout=(30, 30),)
#         soup = BeautifulSoup(response.text, 'html.parser')

#         for label in metadata_labels:
#             elements = soup.find_all(string=lambda text: text and label in text)  
#             if elements:
#                 last_element = elements[-1]
#                 value = last_element.find_next().text.strip()

#                 if 'function' not in value:
#                     metadata.at[indx, label] = value
#                     print(f'Added {label} {indx} {url}')
    
#     except requests.exceptions.RequestException as e:
        
#         failed_requests.append(indx)
#         print(f'Failed {indx} {url}\n{e}\n')
    

In [244]:
# print(len(failed_requests))
# failed_requests[:10]

In [245]:
# # # metadata.to_csv('datasets/production/metadata_idrecord_GSD.csv', sep=';', encoding='utf-8', index=False)

Add all scrapped metadata back to big DataFrame `df_images`

In [246]:
# <----- Resulting dataframe ----->
metadata = pd.read_csv('datasets/production/metadata_idrecord_GSD.csv', sep=';', encoding='utf-8')
metadata.fillna(np.nan, inplace=True)

In [247]:
df_images = df_images.merge(metadata[["id_record", "Gattung", "Standort", "Datierung"]], on="id_record", how="left")
df_images.head(3)

,id_record,id_persistent,label_id,image_links,archive_links,microfiche_links,microfiche_archive_links,Gattung,Standort,Datierung
0,http://www.bildindex.de/document/obj20666124,http://id.bildindex.de/thing/0001618283,0,[],[],['http://www.bildindex.de/bilder/d/mi12308g07'],['https://www.bildindex.de/media/obj20666124/m...,Fotografie (Objektgattung),NaN,1950
1,http://www.bildindex.de/document/obj20727600,http://id.bildindex.de/thing/0001677732,1,['http://www.bildindex.de/bilder/d/fm140030'],['https://www.bildindex.de/media/obj20727600/f...,['http://www.bildindex.de/bilder/d/mi02117a05'...,['https://www.bildindex.de/media/obj20727600/m...,Angewandte Kunst,"Ronhausen, Bauernhaus, 14",1701/1800
2,http://www.bildindex.de/document/obj20943303,http://id.bildindex.de/thing/0001803584,2,['http://www.bildindex.de/bilder/d/STMP_II_110...,['https://www.bildindex.de/media/obj20943303/S...,[],[],"Zeichenkunst, Architekturzeichnung",NaN,"1851/1861, Quelle: Wasserzeichen"


In [248]:
# num_filled = df_images[df_images.Gattung.notna() | df_images.Standort.notna() | df_images.Datierung.notna()].shape[0]
# num_filled_buildings = df_images[df_images.id_persistent.isin(df_buildings.id_persistent) & (df_images.Gattung.notna() | df_images.Standort.notna() | df_images.Datierung.notna())].shape[0]

# print(f'Failed requests: {len(failed_requests)} -- {percentage(len(failed_requests), metadata.shape[0])}%')
# print(f'Filled metadata: {num_filled} -- {percentage(num_filled, df_images.shape[0])}%')
# print(f'Filled metadata (among buildings-related): {num_filled_buildings} -- {percentage(num_filled_buildings, df_buildings.shape[0])}%')

In [249]:
# df_images = df_images.rename(columns={'dated': 'Datierung', 'obj_type': 'Gattung', 'location': 'Standort'})
df_images = df_images.rename(columns={'Datierung': 'dated', 'Gattung': 'obj_type', 'Standort': 'location'})

# Investigating  location metadata

In [250]:
num_unfilled = df_images[df_images.obj_type.isna() & df_images.location.isna() & df_images.dated.isna()].shape[0]
num_unfilled_buildings = df_images[df_images.id_persistent.isin(df_buildings.id_persistent) & df_images.obj_type.isna() & df_images.location.isna() & df_images.dated.isna()].shape[0]
num_filled = df_images.shape[0] - num_unfilled
num_filled_buildings = df_buildings.shape[0] - num_unfilled_buildings

print(f'Filled metadata: {num_filled} -- {percentage(num_filled, df_images.shape[0])}%')
print(f'Filled metadata (among buildings-related by clustering): {num_filled_buildings} -- {percentage(num_filled_buildings, df_buildings.shape[0])}%')

Filled metadata: 34703 -- 99.974%
Filled metadata (among buildings-related by clustering): 8818 -- 99.955%


In [251]:
loc_marburg_unique = df_images[df_images.location.notna() & df_images.location.str.contains('Marburg, ')].location.unique()

print('Addresses in Marburg found:', loc_marburg_unique.shape[0])
print('Examples:')
loc_marburg_unique

Addresses in Marburg found: 1796
Examples:


array(['Marburg, Uferstraße 10b', 'Marburg, Deutschhausstraße',
       'Marburg, Elisabethkirche, Westfassade', ...,
       'Marburg, Wehrda (Marburg), Evangelische Kirche, Chor',
       'Marburg, Am Grün 35', 'Marburg, Am Grün 35a'], dtype=object)

In [252]:
_ = df_images[df_images.location.notna() & df_images.location.isin(loc_marburg_unique)]
print(f'Objects with relevant location (Marburg): {len(_)} ({percentage(len(_), df_images.shape[0])}%)')

_ = _[_.id_record.isin(df_buildings.id_record)]
print(f'Objects with relevant location (Marburg) among buildings related by clustering: {len(_)} ({percentage(len(_), df_buildings.shape[0])}%)')

Objects with relevant location (Marburg): 3984 (11.477%)
Objects with relevant location (Marburg) among buildings related by clustering: 2434 (27.59%)


### Comparison with our list of uni-addresses

In [253]:
addresses_meta = [re.sub('Marburg, ', '', loc) for loc in loc_marburg_unique.tolist()]

In [254]:
with open('datasets/production/buildings_dict.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
uni_addresses = []

for area in data:
    for campus in data[area]:
        for building in data[area][campus]:
            uni_addresses.append(building['address'])

Addresses in the list have structure STREET_NAME HOUSE_NUMBER and (optionaly) slash and additional number. Metadata's addresses are a bit more sophisticated. Another problem is that we don't know if addresses in metadata are contemporary ones or outdated.

We will try to find at least some overlappings and to search:
1. the street name of original address in metadata address
2. if succeed, compare numbers

House number like 2/3 is a bit neglected


In [255]:
uni_streets = [re.sub(r'[0-9/]', '', address) for address in uni_addresses]
uni_streets = [re.sub(r'\s[a-e+]', '', street).strip() for street in uni_streets]
uni_streets = list(set(uni_streets))

# <-- Rules -->
# 2/3 --> 2/3,2,3
# 2/3/4 --> 2,3,4
# 2-4 --> 2,3,4
# 2,5 --> 2,5
# 21a --> 21a
# 4 1/2 --> 4,1/2,1,2

def extract_house_number(address):
    ind_housenumber = re.search('\d', address)
    try:
        house_number = re.sub(r'(\d)\s(\d)', r'\1,\2', address[ind_housenumber.span()[0]:])
        house_number = re.sub(' ','',house_number)
        try:
            return [house_number]
        except:
            houses = []
            for i in re.findall(r'\d{1,2}/\d', house_number):
                houses.append(i)

            house_number = re.sub(r'/',',',house_number).split(',')
            for i in house_number:
                if '-' not in i:
                    houses.append(i)
            
            house_number = house_number[0]
            if '-' in house_number:
                house_number = house_number.split('-')
                house_number = list(range(int(house_number[0]),int(house_number[1])+1))
                for i in house_number:
                    houses.append(i)

            return [str(house) for house in houses]
    except:
        return None

def is_uni_address(address:str):
    is_uni_street = [uni_street 
                     for uni_street in uni_streets 
                     if uni_street+' ' in address or 
                     ((not address.isdigit()) and address in uni_street)]
    if not is_uni_street:
        return False
    street = is_uni_street[0]
    houses = extract_house_number(address)
    if houses == None:
        return False
        # if street in uni_addresses:
        #     return True
        # else:
        #     return False
    options = [street+' '+house for house in houses if street+' '+house in uni_addresses]
    return bool(len(options))

example = 'Lahntor 2/3'
is_uni_address(example)


False

In [256]:
relevant_addresses = []
for address in addresses_meta:
    if is_uni_address(address):
        relevant_addresses.append(address)

print(f'Buildings found from uni-addresses in dataset by location from metadata: {len(relevant_addresses)}')
print('These are', np.sort(np.array(relevant_addresses)))

Buildings found from uni-addresses in dataset by location from metadata: 37
These are ['Bahnhofstraße 7' 'Barfüßerstraße 1' 'Biegenstraße 10' 'Biegenstraße 11'
 'Biegenstraße 12' 'Biegenstraße 14' 'Biegenstraße 9'
 'Deutschhausstraße 10' 'Deutschhausstraße 12' 'Deutschhausstraße 15'
 'Deutschhausstraße 17a' 'Deutschhausstraße 17b' 'Deutschhausstraße 3'
 'Deutschhausstraße 9'
 'Ehemaliges Hygieneinstitut & Pharmakologisches Institut, Pilgrimstein 2'
 'Gutenbergstraße 18' 'Hans-Meerwein-Straße 8'
 'Institut für Leibesübungen, Barfüßerstraße 1' 'Ketzerbach 11'
 'Ketzerbach 63' 'Lahntor 3' 'Landgraf-Philipp-Straße 4' 'Marbacher Weg 6'
 'Pilgrimstein 2' 'Renthof 5' 'Renthof 6' 'Robert-Koch-Straße 4'
 'Robert-Koch-Straße 6' 'Roter Graben 10' 'Schulstraße 12'
 'Universitäts-Kinderklinik, Deutschhausstraße 12' 'Universitätsstraße 24'
 'Universitätsstraße 25' 'Universitätsstraße 6' 'Universitätsstraße 7'
 'Wilhelm-Röpke-Straße 4' 'Wilhelm-Röpke-Straße 6']


There are a couple mistakes:
- `Marburg, Universitäts-Kinderklinik, Deutschhausstraße 12` and `Marburg, Deutschhausstraße 12`
- `Marburg, Barfüßerstraße 1` and `Marburg, Institut für Leibesübungen, Barfüßerstraße 1`
- `Marburg, Ehemaliges Hygieneinstitut & Pharmakologisches Institut, Pilgrimstein 2` and `Marburg, Pilgrimstein 2`

We have to merge them

In [257]:
mistaken_addresses = ['Marburg, Universitäts-Kinderklinik, Deutschhausstraße 12',
                      'Marburg, Institut für Leibesübungen, Barfüßerstraße 1',
                      'Marburg, Ehemaliges Hygieneinstitut & Pharmakologisches Institut, Pilgrimstein 2']

df_images['location_additional'] = None

for address in mistaken_addresses:
    correct_address = ','.join([address.split(',')[0], address.split(',')[2]])
    df_images.loc[df_images.location == address, 'location_additional'] = address.split(',')[1]
    df_images.loc[df_images.location == address, 'location'] = correct_address

for address in relevant_addresses:
    if 'Marburg, ' + address in mistaken_addresses:
        relevant_addresses.remove(address)
    
print(f'Buildings found from uni-addresses in dataset by location from metadata: {len(relevant_addresses)}')

Buildings found from uni-addresses in dataset by location from metadata: 34


There are actually 34, since three of them are copy of each other with additional information. We manually found 34 buildings related to University in Google Maps, so this `34` is sufficient

Let's try to choose only objects with relevant addresses (uni address)

In [258]:
# Add 'Marburg, ' back to location metadata to select relevant rows in dataset with metadata
relevant_addresses = ['Marburg, '+a for a in relevant_addresses]

In [259]:
ind_uni_objects = df_images[df_images.location.isin(relevant_addresses)].index

print(f'Univesity related buildings with number of objects (Total {ind_uni_objects.shape[0]}) associated:')

df_images[df_images.index.isin(ind_uni_objects)].groupby('location').size().sort_values(ascending=False).head(10)

Univesity related buildings with number of objects (Total 72) associated:


location
Marburg, Biegenstraße 11              10
Marburg, Lahntor 3                     9
Marburg, Barfüßerstraße 1              6
Marburg, Deutschhausstraße 10          6
Marburg, Pilgrimstein 2                3
Marburg, Deutschhausstraße 12          3
Marburg, Ketzerbach 63                 2
Marburg, Landgraf-Philipp-Straße 4     2
Marburg, Gutenbergstraße 18            2
Marburg, Deutschhausstraße 3           2
dtype: int64

Let's find related images in the whole MVP dataset

In [260]:
ind_uni_objects = [str(ind) for ind in ind_uni_objects]

ind_uni_images = df_mvp.id_objects[df_mvp.id_objects.apply(lambda lst: any(x in ind_uni_objects for x in lst))].index

print('Number of univesity images:')
print('  Outdoor:', (df_mvp.loc[ind_uni_images, 'cluster_type'] == 'outdoor').sum())
print('  Indoor:', (df_mvp.loc[ind_uni_images, 'cluster_type'] == 'indoor').sum())
print('  People:', (df_mvp.loc[ind_uni_images, 'cluster_type'] == 'people').sum())
print('  Details:', (df_mvp.loc[ind_uni_images, 'cluster_type'] == 'details').sum())

Number of univesity images:
  Outdoor: 631
  Indoor: 396
  People: 35
  Details: 45


In [261]:
print('Number of images for each uni-address:')

grouped_indices = (
    df_images.iloc[ind_uni_objects]
    .groupby('location')['location']
    .apply(lambda x: list(x.index))
)

images_per_location = []

for loc, inds in grouped_indices.items():
    links_images = []
    for ind in inds:
        links_images.append(df_mvp[df_mvp.id_objects.apply(lambda lst: str(ind) in lst)].link.unique())
    links_images = np.unique(np.concatenate(links_images))
    images_per_location.append((loc, links_images))
    # print(f'  {len(links_images)}\t{loc}')

images_per_location = sorted(images_per_location, key=lambda x: len(x[1]), reverse=True)

for loc, links in images_per_location:
    print(f'  {len(links)}\t{loc}')

Number of images for each uni-address:
  396	Marburg, Biegenstraße 11
  318	Marburg, Deutschhausstraße 9
  195	Marburg, Lahntor 3
  39	Marburg, Wilhelm-Röpke-Straße 6
  37	Marburg, Biegenstraße 14
  36	Marburg, Deutschhausstraße 10
  31	Marburg, Biegenstraße 10
  30	Marburg, Deutschhausstraße 17a
  23	Marburg, Wilhelm-Röpke-Straße 4
  21	Marburg, Deutschhausstraße 12
  20	Marburg, Barfüßerstraße 1
  19	Marburg, Robert-Koch-Straße 4
  11	Marburg, Deutschhausstraße 3
  10	Marburg, Biegenstraße 12
  10	Marburg, Ketzerbach 63
  7	Marburg, Landgraf-Philipp-Straße 4
  7	Marburg, Pilgrimstein 2
  7	Marburg, Renthof 5
  7	Marburg, Universitätsstraße 7
  6	Marburg, Gutenbergstraße 18
  6	Marburg, Marbacher Weg 6
  6	Marburg, Universitätsstraße 6
  5	Marburg, Bahnhofstraße 7
  5	Marburg, Deutschhausstraße 17b
  4	Marburg, Deutschhausstraße 15
  4	Marburg, Robert-Koch-Straße 6
  3	Marburg, Universitätsstraße 24
  2	Marburg, Hans-Meerwein-Straße 8
  2	Marburg, Ketzerbach 11
  2	Marburg, Schulstraß

In [262]:
relevant_addresses = sorted(relevant_addresses)
# relevant_addresses

#### Show all image links for one address

In [263]:
address = relevant_addresses[3]

print(f'Address: {address}\nImages:')

links = []
for ind in grouped_indices[address]:
    links.append(df_mvp[df_mvp.id_objects.apply(lambda lst: str(ind) in lst)].link)
for link in np.unique(np.concatenate(links)):
    print(link)

Address: Marburg, Biegenstraße 11
Images:
http://www.bildindex.de/bilder/d/fm1351620
http://www.bildindex.de/bilder/d/fm1351621
http://www.bildindex.de/bilder/d/fm1351622
http://www.bildindex.de/bilder/d/fm1351623
http://www.bildindex.de/bilder/d/fm141330
http://www.bildindex.de/bilder/d/fm15243
http://www.bildindex.de/bilder/d/fm15245
http://www.bildindex.de/bilder/d/fm193412
http://www.bildindex.de/bilder/d/fm21211
http://www.bildindex.de/bilder/d/fm21212
http://www.bildindex.de/bilder/d/fm21224
http://www.bildindex.de/bilder/d/fm222685
http://www.bildindex.de/bilder/d/fm3000043
http://www.bildindex.de/bilder/d/fm3000044
http://www.bildindex.de/bilder/d/fm3000045
http://www.bildindex.de/bilder/d/fm3000046
http://www.bildindex.de/bilder/d/fm3000047
http://www.bildindex.de/bilder/d/fm431121
http://www.bildindex.de/bilder/d/fm431694
http://www.bildindex.de/bilder/d/fm431872
http://www.bildindex.de/bilder/d/fm431873
http://www.bildindex.de/bilder/d/fm497910
http://www.bildindex.de/bilder

# Investigating date metadata

We can't retrieve any useful information about (ir)relevancy of images from dates, but this information might be helpful in further story creation, like orginising images of one building by its dates in hystorical order. 

In [264]:
dates = df_images.dated
dates_uni = df_images.iloc[ind_uni_objects].dated

print(f'Objects with date: {dates.notna().sum()} ({percentage(dates.notna().sum(), df_images.shape[0])}%)')
print(f'Objects with date among uni-objects: {dates_uni.notna().sum()} ({percentage(dates_uni.notna().sum(), df_images.iloc[ind_uni_objects].shape[0])}%)')
print('Semiunique dates found:', dates.unique().shape[0])

Objects with date: 29785 (85.806%)
Objects with date among uni-objects: 70 (97.222%)
Semiunique dates found: 5217


Let's see what those dates are:

In [265]:
for year in dates_uni.unique()[:5]:
    print(year)
print('...')

1883-1887 Erweiterung: 1900 Zerstörung: 1944 Wiederaufbau: 1947-1951
1870-1878 Erweiterung: 1887-1891 Restaurierung: 1945/1946 Umbau: 1962-1966
1964-1967 Einweihung: 1967.11.18
1899-1902
1853-1858
...


#### Scraping dates on `id_persistent` for uni-buildings only (only if you don't have `dates_uni_images.csv`)

In [266]:
# dates_uni_images = pd.DataFrame(columns=['date_taken', 'id','link_obj'])
# dates_uni_images['id'] = df_mvp.id
# dates_uni_images['link_obj'] = df_images.id_record.iloc[df_mvp.id_objects.apply(lambda lst: lst[0])].to_list()
# dates_uni_images = dates_uni_images.iloc[ind_uni_images]

In [267]:
# for indx in dates_uni_images.index:
#     if indx not in failed_requests:
#         continue

#     id_obj = re.findall('obj\d+', dates_uni_images.link_obj.loc[indx])[0]
#     url = 'https://www.bildindex.de/media/' + id_obj + '/' + dates_uni_images.id.loc[indx]
#     is_filled = not pd.isnull(dates_uni_images.loc[indx].date_taken)

#     if is_filled:
#         continue

#     try:

#         response = requests.get(url, timeout=(30, 30),)
#         soup = BeautifulSoup(response.text, 'html.parser')

#         label = 'Aufnahmedatum'
#         elements = soup.find_all(string=lambda text: text and label in text)  

#         if elements:
#             last_element = elements[-1]
#             value = last_element.find_next().text.strip()
#             if 'function' not in value:
#                 dates_uni_images.at[indx, 'date_taken'] = value
#                 print(f'Added {label} {indx} {url}')
    
#     except requests.exceptions.RequestException as e:
        
#         failed_requests.append(indx)
#         print(f'Failed {indx} {url}\n{e}\n')

In [268]:
# # # # dates_uni_images.to_csv('datasets/production/dates_uni_images.csv', sep=';', encoding='utf-8', index=True, index_label='my_index')

In [269]:
# <----- Resulting dataframe ----->
dates_uni_images = pd.read_csv('datasets/production/dates_uni_images.csv', sep=';', encoding='utf-8', index_col=0)
dates_uni_images.head(4)

,date_taken,id,link_obj
my_index,,,
342,um 1895/1920?,http://www.bildindex.de/bilder/d/fm1048507,http://www.bildindex.de/document/obj20269564
362,um 1895/1920?,http://www.bildindex.de/bilder/d/fm1089589,http://www.bildindex.de/document/obj20269564
861,1953/1961,http://www.bildindex.de/bilder/d/fm1350906,http://www.bildindex.de/document/obj20269774
864,nach 1961,http://www.bildindex.de/bilder/d/fm1350909,http://www.bildindex.de/document/obj20269457


# List university images in hierarchical order

In [270]:
num_unfilled = dates_uni_images.date_taken.isna().sum()
print(f'Images without information about taken date: {num_unfilled} -- {percentage(num_unfilled, dates_uni_images.shape[0])}%')

Images without information about taken date: 252 -- 19.874%


In [271]:
dates_uni_images.id = dates_uni_images.id.apply(lambda x: x[re.search('http://www.bildindex.de/bilder/d/', x).span()[1]:])
df_mvp.drop(columns=['date_taken'], inplace=True, errors='ignore')
df_mvp = df_mvp.merge(dates_uni_images[['id', 'date_taken']], on='id', how="left")

#### Show all information about chosen address

In [305]:
# <---- Choose addresses ---->

ind = relevant_addresses.index('Marburg, Barfüßerstraße 1')
address = relevant_addresses[ind]

In [306]:
df_mvp['date_taken_first_year'] = df_mvp.date_taken.apply(lambda d: 
                                                          re.findall(r'\d{4}', d)[0] + ' ('+d+')' 
                                                          if isinstance(d, str) 
                                                          else None)
df_mvp.fillna({'date_taken_first_year':'Unknown'}, inplace=True)

ind_img_associated = df_images[df_images.location == address].index
_ = df_mvp[df_mvp
          .id_objects
          .apply(lambda lst: 
                 any(int(x) in ind_img_associated for x in lst))].groupby(['date_taken_first_year'])['link'].apply(list)

In [319]:
print(f'Address:\n  {address}\n')

loc_add = df_images[(df_images.location == address) & (df_images.location_additional.notna())].location_additional.unique()[0]
print(f'Additional location:\n  {loc_add}\n')

print(f'Dates associated:')

for info in df_images[df_images.location == address].dated.unique():
    print('  ', info)
print()

print(f'Images associated:')
for i in _.items():
    print('  ', i[0])
    for link in i[1]:
        print('    ', link)

Address:
  Marburg, Barfüßerstraße 1

Additional location:
   Institut für Leibesübungen

Dates associated:
   1731-1732
   1731-1732?
   Gründung: 1234 Teilabbruch: 1723

Images associated:
   1869 (1869/1900)
     http://www.bildindex.de/bilder/d/fm810444
   1920 (um 1920/1930?)
     http://www.bildindex.de/bilder/d/fm1522108
   1930 (um 1930/1940)
     http://www.bildindex.de/bilder/d/fmb8208_18
   1932 (um 1932/1940?)
     http://www.bildindex.de/bilder/d/fm65123
   1961 (um 1961/1969?)
     http://www.bildindex.de/bilder/d/mi02050f14
   1974 (1974)
     http://www.bildindex.de/bilder/d/fmb131_10
     http://www.bildindex.de/bilder/d/fmb133_01
     http://www.bildindex.de/bilder/d/fmb133_08
     http://www.bildindex.de/bilder/d/fmb133_09
     http://www.bildindex.de/bilder/d/mi02050f11
   1976 (1976)
     http://www.bildindex.de/bilder/d/mi02050f12
     http://www.bildindex.de/bilder/d/mi02050f13
   1990 (1990)
     http://www.bildindex.de/bilder/d/fmbc23550_03
     http://www.bild

# Investigating object type metadata

In [289]:
print(f'All possible obj_type in dataset (Total {df_images.obj_type.unique().shape[0]}):')
df_images.obj_type.unique()

All possible obj_type in dataset (Total 178):


array(['Fotografie (Objektgattung)', 'Angewandte Kunst',
       'Zeichenkunst, Architekturzeichnung', 'Zeichenkunst',
       'Druckgraphik', 'Skulptur', 'Malerei', 'Keramik (Objektgattung)',
       'Keramik (Objektgattung), Fayence', 'Architektur', 'Textilkunst',
       'Keramik (Objektgattung), Feinkeramik', 'Seriendruck',
       'Bauskulptur', 'Reisefotografie', 'Literatur', 'Tafelmalerei',
       'vorbereitende Zeichnung', 'Modellbau',
       'Schriftkunst, Zeichenkunst', 'Schriftkunst', 'Kleinskulptur',
       'Dokumentarfotografie (Objektgattung)', 'Sakralraumskulptur',
       'Fotografie (Objektgattung), Panoramafotografie',
       'Angewandte Kunst, Schmiedekunst',
       'Keramik (Objektgattung), Irdenware',
       'Angewandte Kunst, Irdenware', 'Zeichenkunst, Malerei',
       'Angewandte Kunst, Skulptur', 'Einblattdruck', 'Buchkunst',
       'Druckgraphik, Buchillustration', 'Glasmalerei',
       'Keramik (Objektgattung), Steinzeug',
       'Angewandte Kunst, Steinzeug', 'Text

Me might assume that all university-related objects would have 'Architektur' as an obj_type. However picture of university building might be 'Fotografie', 'Zeichenkunst', 'Angewandte Kunst', 'Skulptur' (part of the facade) etc. and not having the address of building (being None).  

With obj_type we can get rid of something definitely irrelevant like 'Keramik'. However, we see other more workable and beneficial ways to improved dataset to come to final result, so we would put investigation of this column for future work on project for other groups  

**Comment** However, clustering might be done after separation of images by obj_type instead of working with the whole dataset

# Saving datasets

In [290]:
df_mvp.head(4)

,link,id_objects,labels,file_name,id,cluster,cluster_type,date_taken,date_taken_first_year
0,http://www.bildindex.de/bilder/d/ae00001b05,[9077],[1617],scn-ae00001b05.jpg,ae00001b05,85,notype,NaN,Unknown
1,http://www.bildindex.de/bilder/d/ae00001b06,[9077],[1617],scn-ae00001b06.jpg,ae00001b06,85,notype,NaN,Unknown
2,http://www.bildindex.de/bilder/d/ae00001b07,[9077],[1617],scn-ae00001b07.jpg,ae00001b07,85,notype,NaN,Unknown
3,http://www.bildindex.de/bilder/d/ae00001b08,[9077],[1617],scn-ae00001b08.jpg,ae00001b08,96,notype,NaN,Unknown


In [ ]:
# df_mvp.to_csv('datasets/production/dataset_MVP_clustered_metadata.csv', sep=';', encoding='utf-8', index=False)

In [304]:
df_images.head(4)

,id_record,id_persistent,label_id,image_links,archive_links,microfiche_links,microfiche_archive_links,obj_type,location,dated,location_additional
0,http://www.bildindex.de/document/obj20666124,http://id.bildindex.de/thing/0001618283,0,[],[],['http://www.bildindex.de/bilder/d/mi12308g07'],['https://www.bildindex.de/media/obj20666124/m...,Fotografie (Objektgattung),NaN,1950,None
1,http://www.bildindex.de/document/obj20727600,http://id.bildindex.de/thing/0001677732,1,['http://www.bildindex.de/bilder/d/fm140030'],['https://www.bildindex.de/media/obj20727600/f...,['http://www.bildindex.de/bilder/d/mi02117a05'...,['https://www.bildindex.de/media/obj20727600/m...,Angewandte Kunst,"Ronhausen, Bauernhaus, 14",1701/1800,None
2,http://www.bildindex.de/document/obj20943303,http://id.bildindex.de/thing/0001803584,2,['http://www.bildindex.de/bilder/d/STMP_II_110...,['https://www.bildindex.de/media/obj20943303/S...,[],[],"Zeichenkunst, Architekturzeichnung",NaN,"1851/1861, Quelle: Wasserzeichen",None
3,http://www.bildindex.de/document/obj20948462,http://id.bildindex.de/thing/0001806337,3,['http://www.bildindex.de/bilder/d/STMP_II_156...,['https://www.bildindex.de/media/obj20948462/S...,[],[],"Zeichenkunst, Architekturzeichnung",NaN,NaN,None


In [ ]:
# df_images.to_csv('datasets/production/df__CLEAN_UPDATED_metadata.csv', sep=';', encoding='utf-8', index=False)